# 🎯 Deep Learning Homework - Hair Type Classification

## Machine Learning Zoomcamp 2025 - Module 8

This notebook implements a CNN for binary classification of hair types (straight vs curly) using PyTorch.

## 1. Setup and Imports

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import wget
import zipfile
import statistics

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "D:\tien.nv12\learn\machine-learning-zoomcamp\.venv\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

## 2. Reproducibility Setup

In [ ]:
# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("✅ Reproducibility seeds set")

## 3. Download and Prepare Dataset

In [ ]:
# Download dataset
url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"

if not os.path.exists("data.zip"):
    print("Downloading dataset...")
    wget.download(url, "data.zip")
    print("\nDownload completed!")

if not os.path.exists("data"):
    print("Extracting dataset...")
    with zipfile.ZipFile("data.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    print("Extraction completed!")
else:
    print("Dataset already exists!")

## 4. Model Definition

In [ ]:
class HairCNN(nn.Module):
    """
    CNN for hair type classification following homework specifications:
    - Input: (3, 200, 200)
    - Conv2d: 32 filters, kernel_size=(3,3), padding=0, stride=1, ReLU
    - MaxPool2d: pool_size=(2,2)
    - Flatten
    - Linear: 64 neurons, ReLU
    - Linear: 1 neuron (for binary classification)
    """
    
    def __init__(self):
        super(HairCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                              kernel_size=3, padding=0, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Calculate the size after conv and pooling
        # Input: (3, 200, 200)
        # After conv1: (32, 198, 198)  # 200-3+1 = 198
        # After pool1: (32, 99, 99)    # 198/2 = 99
        
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        # Fully connected layers
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        return x

print("✅ Model architecture defined")

## 5. Question 1: Loss Function Selection

In [ ]:
print("=" * 50)
print("QUESTION 1: Which loss function to use?")
print("=" * 50)
print("Options:")
print("- nn.MSELoss()")
print("- nn.BCEWithLogitsLoss() ← CORRECT")
print("- nn.CrossEntropyLoss()")
print("- nn.CosineEmbeddingLoss()")
print()
print("For binary classification, we should use:")
print("✓ nn.BCEWithLogitsLoss() - Binary Cross Entropy with Logits")
print("This is appropriate for binary classification tasks.")

answer_q1 = "nn.BCEWithLogitsLoss()"
print(f"\n📝 ANSWER Q1: {answer_q1}")

## 6. Question 2: Count Model Parameters

In [ ]:
print("=" * 50)
print("QUESTION 2: Total number of parameters")
print("=" * 50)
print("Options: 896, 11214912, 15896912, 20073473")
print()

# Create model and count parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HairCNN().to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

# Try to use torchsummary for detailed breakdown
try:
    summary(model, input_size=(3, 200, 200))
except Exception as e:
    print(f"torchsummary error: {e}")

# Manual calculation verification
print("\nManual calculation:")
conv1_params = (3 * 3 * 3 + 1) * 32
fc1_params = (32 * 99 * 99 + 1) * 64
fc2_params = (64 + 1) * 1
print(f"Conv1 parameters: {conv1_params:,}")
print(f"FC1 parameters: {fc1_params:,}")
print(f"FC2 parameters: {fc2_params:,}")
calculated_total = conv1_params + fc1_params + fc2_params
print(f"Total calculated: {calculated_total:,}")

answer_q2 = total_params
print(f"\n📝 ANSWER Q2: {answer_q2:,}")

## 7. Data Transformations and Loaders

In [ ]:
# Initial transforms (without augmentation)
train_transforms_initial = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

# Test transforms (no augmentation)
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load datasets without augmentation
train_dataset = datasets.ImageFolder('data/train', transform=train_transforms_initial)
test_dataset = datasets.ImageFolder('data/test', transform=test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {train_dataset.classes}")
print(f"Class to index mapping: {train_dataset.class_to_idx}")

## 8. Training Setup

In [ ]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

print(f"Using device: {device}")
print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer}")

## 9. Initial Training (10 epochs without augmentation)

In [ ]:
print("=" * 50)
print("INITIAL TRAINING (WITHOUT AUGMENTATION)")
print("=" * 50)

num_epochs = 10
history_initial = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history_initial['loss'].append(epoch_loss)
    history_initial['acc'].append(epoch_acc)

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history_initial['val_loss'].append(val_epoch_loss)
    history_initial['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1:2d}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

print("\n✅ Initial training completed!")

## 10. Question 3: Median Training Accuracy

In [ ]:
print("=" * 50)
print("QUESTION 3: Median of training accuracy for all epochs")
print("=" * 50)
print("Options: 0.05, 0.12, 0.40, 0.84")
print()

print("Training accuracies for all 10 epochs:")
for i, acc in enumerate(history_initial['acc']):
    print(f"Epoch {i+1:2d}: {acc:.4f}")

median_acc = statistics.median(history_initial['acc'])
print(f"\nMedian training accuracy: {median_acc:.4f}")

# Find closest option
options_q3 = [0.05, 0.12, 0.40, 0.84]
closest_q3 = min(options_q3, key=lambda x: abs(x - median_acc))
print(f"Closest option: {closest_q3}")

answer_q3 = closest_q3
print(f"\n📝 ANSWER Q3: {answer_q3}")

## 11. Question 4: Standard Deviation of Training Loss

In [ ]:
print("=" * 50)
print("QUESTION 4: Standard deviation of training loss for all epochs")
print("=" * 50)
print("Options: 0.007, 0.078, 0.171, 1.710")
print()

print("Training losses for all 10 epochs:")
for i, loss in enumerate(history_initial['loss']):
    print(f"Epoch {i+1:2d}: {loss:.4f}")

std_loss = statistics.stdev(history_initial['loss'])
print(f"\nStandard deviation of training loss: {std_loss:.4f}")

# Find closest option
options_q4 = [0.007, 0.078, 0.171, 1.710]
closest_q4 = min(options_q4, key=lambda x: abs(x - std_loss))
print(f"Closest option: {closest_q4}")

answer_q4 = closest_q4
print(f"\n📝 ANSWER Q4: {answer_q4}")

## 12. Data Augmentation Setup

In [ ]:
# Transforms with data augmentation
train_transforms_augmented = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Create new dataset with augmentation
train_dataset_aug = datasets.ImageFolder('data/train', transform=train_transforms_augmented)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)

print("✅ Data augmentation setup completed!")
print("Augmentations applied:")
print("- RandomRotation(50)")
print("- RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1))")
print("- RandomHorizontalFlip()")

## 13. Training with Data Augmentation (10 more epochs)

In [ ]:
print("=" * 50)
print("TRAINING WITH DATA AUGMENTATION")
print("=" * 50)

# Continue training with augmentation (don't recreate model!)
num_epochs_aug = 10
history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

print("Continuing training with data augmentation for 10 more epochs...")

for epoch in range(num_epochs_aug):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1:2d}/{num_epochs_aug}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

print("\n✅ Augmentation training completed!")

## 14. Question 5: Mean Test Loss with Augmentations

In [ ]:
print("=" * 50)
print("QUESTION 5: Mean test loss for all epochs with augmentations")
print("=" * 50)
print("Options: 0.008, 0.08, 0.88, 8.88")
print()

print("Validation losses for all 10 epochs with augmentation:")
for i, loss in enumerate(history_aug['val_loss']):
    print(f"Epoch {i+1:2d}: {loss:.4f}")

mean_val_loss = statistics.mean(history_aug['val_loss'])
print(f"\nMean validation loss: {mean_val_loss:.4f}")

# Find closest option
options_q5 = [0.008, 0.08, 0.88, 8.88]
closest_q5 = min(options_q5, key=lambda x: abs(x - mean_val_loss))
print(f"Closest option: {closest_q5}")

answer_q5 = closest_q5
print(f"\n📝 ANSWER Q5: {answer_q5}")

## 15. Question 6: Average Test Accuracy for Last 5 Epochs

In [ ]:
print("=" * 50)
print("QUESTION 6: Average test accuracy for last 5 epochs (6-10)")
print("=" * 50)
print("Options: 0.08, 0.28, 0.68, 0.98")
print()

print("Validation accuracies for all 10 epochs with augmentation:")
for i, acc in enumerate(history_aug['val_acc']):
    print(f"Epoch {i+1:2d}: {acc:.4f}")

# Get validation accuracy for epochs 6-10 (indices 5-9)
last_5_acc = history_aug['val_acc'][5:]  # epochs 6-10
print(f"\nLast 5 epochs (6-10) validation accuracies:")
for i, acc in enumerate(last_5_acc, 6):
    print(f"Epoch {i:2d}: {acc:.4f}")

avg_last_5_acc = statistics.mean(last_5_acc)
print(f"\nAverage validation accuracy for last 5 epochs: {avg_last_5_acc:.4f}")

# Find closest option
options_q6 = [0.08, 0.28, 0.68, 0.98]
closest_q6 = min(options_q6, key=lambda x: abs(x - avg_last_5_acc))
print(f"Closest option: {closest_q6}")

answer_q6 = closest_q6
print(f"\n📝 ANSWER Q6: {answer_q6}")

## 16. Final Answers Summary

In [ ]:
print("=" * 70)
print("HOMEWORK ANSWERS SUMMARY")
print("=" * 70)
print(f"Question 1: {answer_q1}")
print(f"Question 2: {answer_q2:,}")
print(f"Question 3: {answer_q3} (actual: {median_acc:.4f})")
print(f"Question 4: {answer_q4} (actual: {std_loss:.4f})")
print(f"Question 5: {answer_q5} (actual: {mean_val_loss:.4f})")
print(f"Question 6: {answer_q6} (actual: {avg_last_5_acc:.4f})")
print("=" * 70)
print()
print("🎯 Submit your answers here:")
print("https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw08")

## 17. Model Performance Analysis

In [ ]:
# Create performance plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training and Validation Loss (Initial)
axes[0, 0].plot(range(1, 11), history_initial['loss'], 'b-', label='Training Loss')
axes[0, 0].plot(range(1, 11), history_initial['val_loss'], 'r-', label='Validation Loss')
axes[0, 0].set_title('Loss - Initial Training (No Augmentation)')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Plot 2: Training and Validation Accuracy (Initial)
axes[0, 1].plot(range(1, 11), history_initial['acc'], 'b-', label='Training Accuracy')
axes[0, 1].plot(range(1, 11), history_initial['val_acc'], 'r-', label='Validation Accuracy')
axes[0, 1].set_title('Accuracy - Initial Training (No Augmentation)')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Plot 3: Training and Validation Loss (Augmentation)
axes[1, 0].plot(range(1, 11), history_aug['loss'], 'b-', label='Training Loss')
axes[1, 0].plot(range(1, 11), history_aug['val_loss'], 'r-', label='Validation Loss')
axes[1, 0].set_title('Loss - Training with Augmentation')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Plot 4: Training and Validation Accuracy (Augmentation)
axes[1, 1].plot(range(1, 11), history_aug['acc'], 'b-', label='Training Accuracy')
axes[1, 1].plot(range(1, 11), history_aug['val_acc'], 'r-', label='Validation Accuracy')
axes[1, 1].set_title('Accuracy - Training with Augmentation')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print("📈 Key Observations:")
print("1. Initial training showed overfitting (high train acc, lower val acc)")
print("2. Data augmentation reduced overfitting")
print("3. Validation accuracy improved with augmentation")
print("4. Training and validation curves became more aligned with augmentation")